In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests
import time
from bs4 import BeautifulSoup

url = 'https://www.tldep.gov.taipei/Public/EnvMonitoring/River_AutoStation_Intro.aspx'

driver = webdriver.Chrome('./chromedriver')
driver.get(url)
s = Select(driver.find_element_by_name('ctl00$CPH_Content$Ddl_site'))

#從下拉式選單找出所有option(測站選項)
stnList=[]
for st in s.options:         
    stnList.append(st.text)
print(stnList)

#發布時間
soup = BeautifulSoup(driver.page_source, 'html.parser') 
dt = soup.find('dl', {'class':'PublishDate'}).find('dt')
date = soup.find('dl', {'class':'PublishDate'}).find('dd') 
date = date.text
print(dt.text,date)    

#水質項目
item = soup.find('div', {'class':'DataCards'}).findAll('dt')
itemList=[]
for t in item:
    itemList.append(t.text)

#單位    
unit = soup.findAll('span', {'class':'Unit'})
unitList=[]
for u in unit:
    unitList.append(u.text)
    
#建立字典，數據和單位存於此
data_dict={}  
data_dict['unit'] = unitList


driver.close()


for stn in stnList:
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    s = Select(driver.find_element_by_name('ctl00$CPH_Content$Ddl_site'))
    
    s.select_by_visible_text(stn) #從下拉式選單選擇測站
    time.sleep(1) #暫停
    
    print('----------------------')
    print(stn)

    soup = BeautifulSoup(driver.page_source, 'html.parser')   
    dataList = soup.find('div', {'class':'DataCards'}).findAll('dd')
    dl = []      #暫存數據的list
    for d in dataList:
        lst = d.text.split()   #data數值和單位分開
        print(lst[0],lst[1])
        dl.append(lst[0])
    data_dict[stn] = dl         #將data list存入字典
    
    driver.close()

print('-----------------------')
print(data_dict)

['成美長壽橋', '承德橋', '臺北橋', '中正']
發布時間： 
2020/11/24 11:00
----------------------
成美長壽橋
13.94 V
0.68 m
6.92 mg/L
206 μS/cm
0.54 mg/L
7.08 --
8.40 mg/L
21 ℃
----------------------
承德橋
11.74 V
1.31 m
5.96 mg/L
155 μS/cm
0.30 mg/L
6.85 --
36.61 mg/L
21 ℃
----------------------
臺北橋
13.97 V
0.46 m
1.97 mg/L
5122 μS/cm
--- mg/L
6.87 --
45.01 mg/L
23 ℃
----------------------
中正
13.75 V
0.27 m
4.53 mg/L
164 μS/cm
2.18 mg/L
7.11 --
8.33 mg/L
22 ℃
-----------------------
{'unit': ['V', 'm', 'mg/L', 'μS/cm', 'mg/L', '--', 'mg/L', '℃'], '成美長壽橋': ['13.94', '0.68', '6.92', '206', '0.54', '7.08', '8.40', '21'], '承德橋': ['11.74', '1.31', '5.96', '155', '0.30', '6.85', '36.61', '21'], '臺北橋': ['13.97', '0.46', '1.97', '5122', '---', '6.87', '45.01', '23'], '中正': ['13.75', '0.27', '4.53', '164', '2.18', '7.11', '8.33', '22']}


In [2]:
import pandas as pd

df = pd.DataFrame(data_dict, index = itemList)
print(df)

print(date)

yyyy = date.split()[0].split('/')[0]
mm = date.split()[0].split('/')[1]
dd = date.split()[0].split('/')[2]
HH = date.split()[1].split(':')[0]
MM = date.split()[1].split(':')[1]

df.to_excel('{}_{}_{}_{}_{}.xlsx'.format(yyyy,mm,dd,HH,MM))
print('finished')

             unit  成美長壽橋    承德橋    臺北橋     中正
 DC電壓           V  13.94  11.74  13.97  13.75
 DEPTH量測深度      m   0.68   1.31   0.46   0.27
 DO溶氧        mg/L   6.92   5.96   1.97   4.53
 EC導電度      μS/cm    206    155   5122    164
 NH3-N氨氮     mg/L   0.54   0.30    ---   2.18
 pH酸鹼度         --   7.08   6.85   6.87   7.11
 SS懸浮固體      mg/L   8.40  36.61  45.01   8.33
 WTEMP水溫        ℃     21     21     23     22

2020/11/24 11:00
finished
